In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import ipywidgets

In [2]:
import re
dict_file = '../data/survey/7041-1996-2015_City_Survey_DataSF.xlsx'
data_dict = pd.read_excel(dict_file, 0, skiprows=3)
data_dict['Variable'] = data_dict['Variable'].str.replace('\n', '') 
data_dict = data_dict.set_index('Variable')

#keep only questions which have FDCBA grades
data_dict1 = data_dict.ix[['id', 'year', 'finweigh', 'zipcode', 'district', 'region']]
data_dict2 = data_dict[data_dict['Value Labels'].str.contains('F-Failing', flags=re.IGNORECASE).fillna(False)]
# remove column 'swcndnbd' as it's not present in data csv
data_dict2 = data_dict2.drop('swcndnbd')
data_dict = pd.concat([data_dict1, data_dict2])

In [3]:
survey_df = pd.read_csv('../data/survey/San_Francisco_City_Survey_Data_1996-2015.csv')

# keep only valid cols
survey_df = survey_df[list(set(data_dict.index) & set(survey_df.columns))]

# remove values of 6 and 7 (dont know, not provided responses)
replace_dict = {col: {6: np.nan, 7: np.nan} for col in data_dict2.index}
survey_df = survey_df.replace(replace_dict)

# consider data only from 2009 - 2015
survey_df = survey_df[survey_df['year'].isin([2009, 2011, 2013, 2015])]

# consider only 941XX zip codes for SF city
survey_df = survey_df[(survey_df['zipcode'] > 94100) & (survey_df['zipcode'] < 94200)]
print(survey_df.shape)

(10271, 60)


/home/sylvain/focus/ipython/ipython/IPython/core/interactiveshell.py:2719: DtypeWarning: Columns (181,183,214) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
cat_qs_counts = data_dict2['Category'].value_counts()
cat_qs_counts

Recreation and Parks     14
City Services General    11
Infrastructure           11
Libraries                 9
Transportation            8
Children and Families     1
Name: Category, dtype: int64

In [5]:
weights = [cat_qs_counts[data_dict2.ix[col, 'Category']] for col in data_dict2.index]
weight_mat = survey_df[data_dict2.index].notnull() * weights
satisfaction_scores = (weight_mat * survey_df[data_dict2.index]).sum(axis=1) / weight_mat.sum(axis=1)
scores_by_zip = satisfaction_scores.groupby(survey_df['zipcode']).mean()

In [6]:
scores_by_zip.sort_values()

zipcode
94133    3.241308
94130    3.251165
94134    3.316474
94124    3.338728
94109    3.346500
94102    3.350423
94105    3.364550
94121    3.367709
94103    3.368794
94132    3.372073
94122    3.373936
94112    3.389812
94131    3.396762
94126    3.401733
94110    3.404274
94118    3.412923
94127    3.429824
94115    3.444461
94107    3.447126
94117    3.467985
94116    3.468903
94125    3.469477
94108    3.474260
94111    3.479376
94114    3.482407
94101    3.483300
94158    3.484634
94119    3.495446
94123    3.501281
94106    3.520606
94120    3.529241
94113    3.530031
94104    3.547924
94129    3.583616
dtype: float64

In [7]:
from ipyleaflet import *

In [8]:
m = Map(center=[37.7749, -122.34580993652344], zoom=12, layout=Layout(height='600px'))

In [18]:
import matplotlib as mpl
import matplotlib.cm
import matplotlib.colors
import numpy as np

def n_colors(n, colormap=mpl.cm.Blues):
    data = np.linspace(0.0,1.0,n)
    c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(data)]
    return c

def data_to_colors(data, colormap=mpl.cm.plasma):
    c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(mpl.colors.Normalize()(data))]
    return c

In [19]:
zips = [str(int(i)) for i in scores_by_zip.index]

In [20]:
colors = data_to_colors(scores_by_zip.values)

In [21]:
cols = {}
for i in range(len(zips)):
    cols[zips[i]] = colors[i]

In [22]:
import json
with open('./../mapdata/sf_zipcodes.geojson') as f:
    data = json.load(f)

In [23]:
for feature in data['features']:
    feature['properties']['style'] = {
        'color':cols[feature['id']],
        'weight': 1,
        'fillColor':cols[feature['id']],
        'fillOpacity':0.75,
    }

In [24]:
g = GeoJSON(data=data)

In [25]:
m += g

In [26]:
m

In [21]:
def click_handler(event=None, id=None, properties=None):
    print(properties, event, id)

g.on_click(click_handler)